In [1]:
import json
from datetime import datetime
import ast
import sys
import pickle
import pandas as pd
import numpy as np
from utility import *
import geopandas as gpd
pd.options.mode.chained_assignment = None  # default='warn'
import compute_rhino3d.Util
import compute_rhino3d.Grasshopper as gh
compute_rhino3d.Util.authToken = ""
compute_rhino3d.Util.url = "http://localhost:8081/"
#compute_rhino3d.Util.url = "http://13.250.102.40:80/"
#compute_rhino3d.Util.apiKey = '0hOfevzxs49OfbXDqyUx'

from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.factory import get_sampling, get_crossover, get_mutation
from pymoo.factory import get_termination
from pymoo.optimize import minimize
from pymoo.config import Config
Config.show_compile_hint = False

# load database
rn_gdf = gpd.read_file('static\geojson\Clean_MP19_RoadNetwork_Tampines.geojson').to_crs(3857)
lu_gdf = gpd.read_file('static\geojson\Clean_MP19_LandUse_Tampines.geojson').to_crs(3857)

# json input as dict variable
#user_input = ast.literal_eval(sys.argv[1])

lu_id = 'LU_13'
user_input = {
    'Parcel_ID': lu_id,
    'GenCount': 5,
    'PopCount': 5,
    'ParameterBounds': {
        'BKeyXScale': [0.3,0.5],
        'BKeyYscale': [0.6,0.8],
        'GridAngle': [0.0,90.0],
        'GridSpacing':[10.0,20.0],
        'ParcelStoreyScale': [0.2,1.0]
    },
    'ObjectiveWeights': [0.3,0.7]
}

# get relevent data about selected parcel in json format
selected_parcel_json = json.loads(lu_gdf.loc[lu_gdf.UD_ID == user_input['Parcel_ID']].to_json())['features'][0]
selected_parcel_json['properties']['UD_EdgeCategory'], selected_parcel_json['properties']['UD_EdgeClosestPoint'] = GetEdgeCategory(user_input['Parcel_ID'],rn_gdf, lu_gdf)

class TampinesMOOProblem(ElementwiseProblem):
    def __init__(self):
        super().__init__(n_var=5,
                         n_obj=2,
                         n_constr=0,
                         xl=np.array([user_input['ParameterBounds'][key][0] for key in user_input['ParameterBounds'].keys()]),
                         xu=np.array([user_input['ParameterBounds'][key][1] for key in user_input['ParameterBounds'].keys()]))
    def _evaluate(self, x, out, *args, **kwargs):

        # run parametric model, module B
        filename = 'static\gh\module B_showcase.ghx'
        parameters = [json.dumps(selected_parcel_json)] + list(x)
        ids = ["ParcelJSON"] + list(user_input['ParameterBounds'].keys())
        output = EvaluateGrasshopper(filename, parameters, ids)

        if output['RH_OUT:BuildingJSON']['{0}']:
            # run simulation model, module C
            filename = 'static\gh\module C_showcase.ghx'
            parameters = [json.loads(output['RH_OUT:BuildingJSON']['{0}'][0]['data'])]
            ids = ["BuildingJSON"]
            output = EvaluateGrasshopper(filename, parameters, ids)
            fitness_scores = [float(value['{0}'][0]['data']) for key,value in output.items()]

            out["F"] = fitness_scores
            out["G"] = []
        else:
            out["F"] = [9999,9999]
            out["G"] = []

class TampinesSOOProblem(ElementwiseProblem):
    def __init__(self):
        super().__init__(n_var=5,
                         n_obj=1,
                         n_constr=0,
                         xl=np.array([user_input['ParameterBounds'][key][0] for key in user_input['ParameterBounds'].keys()]),
                         xu=np.array([user_input['ParameterBounds'][key][1] for key in user_input['ParameterBounds'].keys()]))
    def _evaluate(self, x, out, *args, **kwargs):

        # run parametric model, module B
        filename = 'static\gh\module B_showcase.ghx'
        parameters = [json.dumps(selected_parcel_json)] + list(x)
        ids = ["ParcelJSON"] + list(user_input['ParameterBounds'].keys())
        output = EvaluateGrasshopper(filename, parameters, ids)

        if output['RH_OUT:BuildingJSON']['{0}'] != []:
            # run simulation model, module C
            filename = 'static\gh\module C_showcase.ghx'
            parameters = [json.loads(output['RH_OUT:BuildingJSON']['{0}'][0]['data'])]
            ids = ["BuildingJSON"]
            output = EvaluateGrasshopper(filename, parameters, ids)
            fitness_scores = [float(value['{0}'][0]['data']) for key,value in output.items()]
            fitness_scores[0] = user_input['ObjectiveWeights'][0] * Remap(fitness_scores[0],0,1.7)
            fitness_scores[1] = user_input['ObjectiveWeights'][1] * Remap(fitness_scores[1],0,4000)
            fitness_scores = np.average(fitness_scores)
            out["F"] = fitness_scores
            out["G"] = []
        else:
            out["F"] = [9999]
            out["G"] = []

# solve MOO or SOO accordingly
if 'ObjectiveWeights' in user_input:
    problem = TampinesSOOProblem()
else:
    problem = TampinesMOOProblem()

res = minimize(
    problem = problem, 
    algorithm = NSGA2(pop_size = int(user_input['PopCount']),n_offsprings = int(int(user_input['PopCount'])/2)), 
    termination = get_termination("n_gen", int(user_input['GenCount'])),
    seed=1,
    save_history=True, 
    verbose = False)
df = ReadResult(res)


In [7]:
if 'ObjectiveWeights' in user_input:
    ndf = df.sort_values(by=['f0']).iloc[:5,:]
    print(ndf)

    gen  pop        x0        x1         x2         x3        x4        f0
20  4.0  0.0  0.383239  0.743903  18.725876  16.980955  0.317405  0.146514
5   1.0  0.0  0.383239  0.743903  18.725876  16.980955  0.317405  0.146514
10  2.0  0.0  0.383239  0.743903  18.725876  16.980955  0.317405  0.146514
15  3.0  0.0  0.383239  0.743903  18.725876  16.980955  0.317405  0.146514
21  4.0  1.0  0.460314  0.787715   0.981536  12.965596  0.901111  0.147041
6   1.0  1.0  0.460314  0.787715   0.981536  12.965596  0.901111  0.147041
16  3.0  1.0  0.460314  0.787715   0.981536  12.965596  0.901111  0.147041
11  2.0  1.0  0.460314  0.787715   0.981536  12.965596  0.901111  0.147041
12  2.0  2.0  0.380213  0.743903  18.725876  17.092710  0.317405  0.151853
22  4.0  2.0  0.380213  0.743903  18.725876  17.092710  0.317405  0.151853
17  3.0  2.0  0.380213  0.743903  18.725876  17.092710  0.317405  0.151853
18  3.0  3.0  0.383235  0.744065   0.010294  12.911571  0.317405  0.151910
23  4.0  3.0  0.383235  0